In [ ]:
#!/usr/bin/python3 -u
import os.path

KEY_FILE = "key"
KEY_LEN = 50000
FLAG_FILE = "flag"


def startup(key_location):
	flag = open(FLAG_FILE).read()
	kf = open(KEY_FILE, "rb").read()

	start = key_location
	stop = key_location + len(flag)

	key = kf[start:stop]
	key_location = stop

	result = list(map(lambda p, k: "{:02x}".format(ord(p) ^ k), flag, key))
	print("This is the encrypted flag!\n{}\n".format("".join(result)))

	return key_location

def encrypt(key_location):
	ui = input("What data would you like to encrypt? ").rstrip()
	if len(ui) == 0 or len(ui) > KEY_LEN:
		return -1

	start = key_location
	stop = key_location + len(ui)

	kf = open(KEY_FILE, "rb").read()

	if stop >= KEY_LEN:
		stop = stop % KEY_LEN
		key = kf[start:] + kf[:stop]
	else:
		key = kf[start:stop]
	key_location = stop

	result = list(map(lambda p, k: "{:02x}".format(ord(p) ^ k), ui, key))

	print("Here ya go!\n{}\n".format("".join(result)))

	return key_location


print("******************Welcome to our OTP implementation!******************")
c = startup(0)
while c >= 0:
	c = encrypt(c)


In [10]:
print('a' * (50000-32))

...


In [11]:
print('a' * 32)

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [ ]:
# Biết trước flag và result
flag = 'a' * 32  # Chuỗi 'a' lặp lại 32 lần
result_hex = "573d1905043d463d1905533d1951033d1958513d195957282b3d19005340323d"

# Chuyển đổi result từ chuỗi thập lục phân sang danh sách các số nguyên
result = [int(result_hex[i:i + 2], 16) for i in range(0, len(result_hex), 2)]
print(result)

# Tìm key bằng cách XOR với mã ASCII của ký tự 'a' (ord('a') = 97)
key = [ord(f) ^ r for f, r in zip(flag, result)]

# # Chuyển key về dạng chuỗi để dễ đọc
# key_chars = ''.join(chr(k) for k in key)

print("Key:", key)


[87, 61, 25, 5, 4, 61, 70, 61, 25, 5, 83, 61, 25, 81, 3, 61, 25, 88, 81, 61, 25, 89, 87, 40, 43, 61, 25, 0, 83, 64, 50, 61]
Key: [54, 92, 120, 100, 101, 92, 39, 92, 120, 100, 50, 92, 120, 48, 98, 92, 120, 57, 48, 92, 120, 56, 54, 73, 74, 92, 120, 97, 50, 33, 83, 92]


In [ ]:
print(key)
# Result đã biết
result_hex = "51466d4e5f575538195551416e4f5300413f1b5008684d5504384157046e4959"

# Chuyển đổi result từ chuỗi thập lục phân sang danh sách các số nguyên
result = [int(result_hex[i:i + 2], 16) for i in range(0, len(result_hex), 2)]
print(result)

# Tìm flag bằng cách XOR với key
flag = ''.join(chr(r ^ key[i]) for i, r in enumerate(result))

print("Flag:", flag)


[54, 92, 120, 100, 101, 92, 39, 92, 120, 100, 50, 92, 120, 48, 98, 92, 120, 57, 48, 92, 120, 56, 54, 73, 74, 92, 120, 97, 50, 33, 83, 92]
[81, 70, 109, 78, 95, 87, 85, 56, 25, 85, 81, 65, 110, 79, 83, 0, 65, 63, 27, 80, 8, 104, 77, 85, 4, 56, 65, 87, 4, 110, 73, 89]
Flag: g*:rda1c1\9+pP{Nd966O


In [40]:
ord('a')

97

In [48]:
from pwn import *

import binascii

offset = 50000 - 32

p = remote('mercury.picoctf.net', 64260)

print(p.recvline())
print(p.recvline())
encrypted_flag = p.recvline().strip()

print(encrypted_flag)

p.recvuntil('?')
p.sendline('a'*offset)

p.recvuntil('?')

p.sendline('a'*32)

p.recvline()

encoded = p.recvline().strip()

print(f'encoded input: {encoded}')

encoded = binascii.unhexlify(encoded)

print(f'unhexed input: {encoded}')
message = 'a'*32

key = []

for e in range(len(encoded)):
    key.append( ord(message[e])^encoded[e] )

print(f'[+] Found key: {key}')
decoded_flag = []

encrypted_flag = binascii.unhexlify(encrypted_flag)

for i in range(32):
    decoded_flag.append( chr(key[i]^encrypted_flag[i]) )

flag = ''.join(decoded_flag)

print(f'flag: {flag}')

[x] Opening connection to mercury.picoctf.net on port 64260
[x] Opening connection to mercury.picoctf.net on port 64260: Trying 18.189.209.142
[+] Opening connection to mercury.picoctf.net on port 64260: Done
b'******************Welcome to our OTP implementation!******************\n'
b'This is the encrypted flag!\n'
b'51466d4e5f575538195551416e4f5300413f1b5008684d5504384157046e4959'


C:\Users\Admin\AppData\Local\Temp\ipykernel_13536\4292590972.py:15: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.recvuntil('?')
C:\Users\Admin\AppData\Local\Temp\ipykernel_13536\4292590972.py:16: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline('a'*offset)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13536\4292590972.py:18: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.recvuntil('?')
C:\Users\Admin\AppData\Local\Temp\ipykernel_13536\4292590972.py:20: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline('a'*32)


encoded input: b'03463d190702003d195004133d190356433d1902503d1950563d1900513d1959'
unhexed input: b'\x03F=\x19\x07\x02\x00=\x19P\x04\x13=\x19\x03VC=\x19\x02P=\x19PV=\x19\x00Q=\x19Y'
[+] Found key: [98, 39, 92, 120, 102, 99, 97, 92, 120, 49, 101, 114, 92, 120, 98, 55, 34, 92, 120, 99, 49, 92, 120, 49, 55, 92, 120, 97, 48, 92, 120, 56]
flag: 3a16944dad432717ccc3945d3d96421a


In [54]:
from pwn import *
p = remote('jupiter.challenges.picoctf.org', 43522)

print(p.recvline())
print(p.recvline())
print(p.recvline())
print(p.recvline())


[x] Opening connection to jupiter.challenges.picoctf.org on port 43522
[x] Opening connection to jupiter.challenges.picoctf.org on port 43522: Trying 3.131.60.8
[+] Opening connection to jupiter.challenges.picoctf.org on port 43522: Done
b'-------------------------------------------------------------------------------\n'
b'nmrogdep ltgt cp fmwg bqdo - bgtjwtrnf_cp_n_mztg_qdhxyd_mobhdwrgdb\n'
b'-------------------------------------------------------------------------------\n'
b'ut utgt rme hwnl hmgt eldr d jwdgetg mb dr lmwg mwe mb mwg plca ecqq ut pdu ltg pcrv, dry eltr c wrytgpemmy bmg elt bcgpe echt ulde udp htdre xf d plca bmwrytgcro cr elt ptd.  c hwpe dnvrmuqtyot c ldy ldgyqf tftp em qmmv wa ultr elt ptdhtr emqy ht plt udp pcrvcro; bmg bgmh elt hmhtre elde eltf gdeltg awe ht crem elt xmde eldr elde c hcole xt pdcy em om cr, hf ltdge udp, dp ce utgt, ytdy ucelcr ht, adgeqf ucel bgcole, adgeqf ucel lmggmg mb hcry, dry elt elmwolep mb ulde udp fte xtbmgt ht.\n'
